In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import time
import numpy as np
import torch

In [3]:
# Load model and tokenizer
model_path = "/content/drive/Shareddrives/517 nlp project/Models/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Ensure the tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

In [4]:
def generate_qa_prompt(question, context):
    prompt = f"""<s>[INST] You are a helpful AI assistant. Read the following context and answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer the question based only on the context provided. [/INST]"""
    return prompt

def normalize_answer(answer):
    """Normalize answer string for comparison"""
    if answer is None:
        return ""
    # Remove punctuation and convert to lowercase
    return str(answer).lower().strip().replace('.', '').replace(',', '')

In [9]:
# Load evaluation dataset
dataset_path = "/content/drive/Shareddrives/517 nlp project/data/2WikiMultihopQA/test.json"
with open(dataset_path, 'r') as f:
    eval_data = json.load(f)

# Limit to 300 samples
eval_data = eval_data[:300]

print(f"Total evaluation examples: {len(eval_data)}")

Total evaluation examples: 300


In [10]:
# Run evaluation
correct_answers = 0
total_questions = len(eval_data)
start_time = time.time()

# ANSI color codes for output formatting
BLUE = '\033[94m'
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
CYAN = '\033[96m'
PURPLE = '\033[95m'
ENDC = '\033[0m'

results = []

for idx, item in enumerate(eval_data):
    question = item['question']
    context = item['original_context']

    prompt = generate_qa_prompt(question, context)

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")
    with torch.no_grad():
        output = model.generate(
            inputs.input_ids,
            max_new_tokens=256,
            temperature=0.3,
            do_sample=True,
            num_beams=4,
            top_p=0.9,
            repetition_penalty=1.2
        )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract model's answer (everything after the instruction)
    model_answer = response.split("[/INST]")[-1].strip()
    normalized_model_answer = normalize_answer(model_answer)

    # Get correct answer
    correct_answer = item.get('answer', '')
    normalized_correct_answer = normalize_answer(correct_answer)

    # Check if answer is correct
    is_correct = normalized_model_answer == normalized_correct_answer
    if is_correct:
        correct_answers += 1

    # Store result
    results.append({
        'question': question,
        'context': context,
        'correct_answer': correct_answer,
        'model_answer': model_answer,
        'is_correct': is_correct
    })

    # Print progress and statistics
    questions_done = idx + 1
    questions_left = total_questions - questions_done
    elapsed_time = time.time() - start_time
    avg_time_per_question = elapsed_time / questions_done
    estimated_time_left = questions_left * avg_time_per_question
    current_accuracy = (correct_answers / questions_done) * 100

    print(f"\n{CYAN}Progress: {questions_done}/{total_questions} questions{ENDC}")
    print(f"{CYAN}Current accuracy: {current_accuracy:.1f}%{ENDC}")
    print(f"{CYAN}Average time per question: {avg_time_per_question:.1f} seconds{ENDC}")
    print(f"{CYAN}Estimated time remaining: {estimated_time_left/60:.1f} minutes{ENDC}\n")

    correct_color = GREEN if is_correct else RED
    print(f"\n{YELLOW}Question {idx}:{ENDC}")
    print(f"{BLUE}Context (truncated):{ENDC}")
    print(f"{BLUE}{context[:200]}...{ENDC}")
    print(f"{BLUE}Question: {question}{ENDC}")
    print(f"{YELLOW}Model answer:{ENDC}")
    print(f"{PURPLE}{model_answer}{ENDC}")
    print(f"{YELLOW}Correct answer:{ENDC}")
    print(f"{PURPLE}{correct_answer}{ENDC}")
    print(f"{correct_color}Correct: {is_correct}{ENDC}")

    # Save intermediate results every 10 questions
    if questions_done % 10 == 0:
        with open('evaluation_results_2wiki_intermediate.json', 'w') as f:
            json.dump(results, f, indent=2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Progress: 1/300 questions
Current accuracy: 100.0%
Average time per question: 1.3 seconds
Estimated time remaining: 6.5 minutes


Question 0:
Context (truncated):
Michael GovanMichael Govan( born 1963) is the director of the Los Angeles County Museum of Art since 2006. Prior to this, Govan worked as the director of the Dia Art Foundation in New York City.
John ...
Question: Michael GovanMichael Govan( born 1963) is the director of the Los Angeles County Museum of Art since 2006. Prior to this, Govan worked as the director of the Dia Art Foundation in New York City.
John DonatichJohn Donatich is the Director of Yale University Press.
Peter LevinPeter Levin is an American director of film, television and theatre.
Ian Barry (director)Ian Barry is an Australian director of film and TV.
John Farrell (businessman)John Farrell is the director of YouTube in Latin America.
One Law for the WomanOne Law for the Woman is a 1924 American silent western film directed by Dell Henderson and starring 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Progress: 2/300 questions
Current accuracy: 50.0%
Average time per question: 3.1 seconds
Estimated time remaining: 15.2 minutes


Question 1:
Context (truncated):
Ralph MurphyRalph Murphy( May 1, 1895 – February 10, 1967) was an American film director. Born in Rockville, Connecticut, Murphy was active in films from 1931 through 1962, with some work in televisio...
Question: Ralph MurphyRalph Murphy( May 1, 1895 – February 10, 1967) was an American film director. Born in Rockville, Connecticut, Murphy was active in films from 1931 through 1962, with some work in television. From 1941 – 44 he was married to Gloria Dickson, whom he directed in" I Want a Divorce". His films include:
W. Augustus BarrattW. Augustus Barratt( 1873- 1947) was a Scottish- born, later American, songwriter and musician.
Claude WeiszClaude Weisz is a French film director born in Paris.
Jacques DécombeJacques Décombe is a French author, actor and director born in 1953.
Fina TorresFina Torres (born Josefina Torres B

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Progress: 7/300 questions
Current accuracy: 14.3%
Average time per question: 6.9 seconds
Estimated time remaining: 33.7 minutes


Question 6:
Context (truncated):
Living Very HappilyLiving Very Happily is Cantopop artist Miriam Yeung's fourteenth Cantonese studio album. It was released by Amusic on 16 October 2009. The album includes eleven new songs. The secon...
Question: Living Very HappilyLiving Very Happily is Cantopop artist Miriam Yeung's fourteenth Cantonese studio album. It was released by Amusic on 16 October 2009. The album includes eleven new songs. The second edition of the album came out on 2 December 2009. The second album includes a DVD with three music videos.
A Town South of BakersfieldA Town South of Bakersfield was a series of three compilation CDs showcasing New Country musicians in the late 1980s and early 1990s. The first album came out in 1986 and featured acts such as Dwight Yoakam. The Lonesome Strangers, and Candye Kane. The second album came out in 1988 and

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Progress: 8/300 questions
Current accuracy: 12.5%
Average time per question: 7.1 seconds
Estimated time remaining: 34.8 minutes


Question 7:
Context (truncated):
Marie of Luxembourg, Queen of FranceMarie of Luxembourg( 1304 – 26 March 1324), was by birth member of the House of Luxembourg and by marriage Queen of France and Navarre. She was the daughter of Henr...
Question: Marie of Luxembourg, Queen of FranceMarie of Luxembourg( 1304 – 26 March 1324), was by birth member of the House of Luxembourg and by marriage Queen of France and Navarre. She was the daughter of Henry VII, Holy Roman Emperor and Margaret of Brabant. Her two siblings were John of Luxembourg and Beatrice of Luxembourg, Queen of Hungary.
Lyon CohenLyon Cohen( 1868–1937) was a Polish- born Canadian businessman and a philanthropist. He was the grandfather of singer/ poet Leonard Cohen.
John WestleyRev. John Westley( 1636 – 78) was an English nonconformist minister. He was the grandfather of John Wesley( founder of Meth

KeyboardInterrupt: 

In [ ]:
# Calculate and display final metrics
final_accuracy = (correct_answers / total_questions) * 100

print("=== Final Evaluation Metrics ===")
print(f"Total examples evaluated: {total_questions}")
print(f"Correct answers: {correct_answers}")
print(f"Accuracy: {final_accuracy:.1f}%")

# Save final results
timestamp = time.strftime("%Y%m%d-%H%M%S")
output_path = f'evaluation_results_2wiki_{timestamp}.json'
with open(output_path, 'w') as f:
    json.dump({
        'metrics': {
            'total_questions': total_questions,
            'correct_answers': correct_answers,
            'accuracy': final_accuracy
        },
        'results': results
    }, f, indent=2)
print(f"\nDetailed results saved to {output_path}")